# Testing — Report

This notebook visualizes the CSV outputs generated by the **Testing** block.

## What this notebook shows
- **Tests without assertion**: Distribution by package (Donut) and top classes (Bar).

> If a CSV is missing or empty, the cell prints an info message and skips the chart.


In [ ]:
# Setup & helpers
# - CSV is read from reports/csv-reports/Testing/Test_Without_Assertion.csv relative to this notebook folder.
# - Minimal console output; only show information if a CSV is missing/empty.
# - Bar charts use an explicit default color so it's easy to tweak later.
# - Titles are standardized without block prefixes.

import os, re
from pathlib import Path
import pandas as pd
import plotly.express as px
from IPython.display import display

pd.set_option('future.no_silent_downcasting', True)

CATEGORY = "Testing"
CSV_BASE = Path("../reports/csv-reports").resolve()
TEST_DIR = CSV_BASE / CATEGORY

# Explicit default color for all bar charts in this notebook
DEFAULT_BAR_COLOR = ["#1f77b4"]

# CSV IO helpers
NA_LITS = ["", " ", "NA", "N/A", "n/a", "NaN", "NULL", "Null", "null", "None", "none", "-", "--"]

def read_csv_safe(p: Path) -> pd.DataFrame:
    """Read a CSV if present; otherwise return an empty DataFrame.
    Prints a minimal info message when missing or unreadable."""
    p = Path(p)
    if not p.exists():
        print(f"[info] Missing CSV: {p}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(p, na_values=NA_LITS, keep_default_na=True)
        df.columns = [str(c).strip() for c in df.columns]
        return df.dropna(how="all")
    except Exception as e:
        print(f"[warn] Failed to read {p}: {e}")
        return pd.DataFrame()

def find_col(df, *cands, default=None, contains=None):
    """Return a column name by exact candidate(s) or substring (case-insensitive)."""
    if df is None or df.empty:
        return default
    low = {c.lower(): c for c in df.columns}
    for c in cands:
        if c and c.lower() in low:
            return low[c.lower()]
    if contains:
        for k, orig in low.items():
            if contains.lower() in k:
                return orig
    return default

def derive_class_and_method(row, class_cols, method_cols, fqn_cols):
    """Try to get classFqn and methodName from multiple possible column shapes."""
    # 1) Prefer explicit columns if present
    cls = None
    for c in class_cols:
        if c and c in row and pd.notna(row[c]) and str(row[c]).strip():
            cls = str(row[c]).strip()
            break
    mtd = None
    for c in method_cols:
        if c and c in row and pd.notna(row[c]) and str(row[c]).strip():
            mtd = str(row[c]).strip()
            break
    # 2) Try to parse from an fqn-like column if needed
    if (cls is None or mtd is None):
        for c in fqn_cols:
            if c and c in row and pd.notna(row[c]):
                s = str(row[c]).strip()
                if "#" in s:
                    parts = s.split("#", 1)
                    cls = cls or parts[0].strip()
                    mtd = mtd or parts[1].strip()
                elif "." in s:
                    tokens = s.split(".")
                    if len(tokens) >= 2:
                        maybe_method = tokens[-1]
                        cls_part = ".".join(tokens[:-1])
                        if re.search(r"\w+\s*\(.*\)", maybe_method):
                            mtd = mtd or maybe_method
                            cls = cls or cls_part
                        else:
                            cls = cls or s
    return cls, mtd

def package_of(class_fqn):
    if not class_fqn or "." not in class_fqn:
        return "(default)"
    return class_fqn.rsplit(".", 1)[0]


## Tests without assertion

In [ ]:
# Charts generated here:
#  - A) Tests without assertion by package (Donut)
#  - B) Top classes with tests lacking assertions (Bar, explicit color)

path = TEST_DIR / "Test_Without_Assertion.csv"
df_raw = read_csv_safe(path)

if df_raw.empty:
    print("[info] No data for Test_Without_Assertion (missing CSV or no rows).")
else:
    class_candidates  = [
        "declaringClass", "class", "owner", "type", "classFqn",
        "TestWithoutAssertion.declaringClass", "TestWithoutAssertion.class", "TestWithoutAssertion.type",
        "TestWithoutAssertion.classFqn", "TestWithoutAssertion.fqn"
    ]
    method_candidates = [
        "methodName", "name", "signature",
        "TestWithoutAssertion.methodName", "TestWithoutAssertion.name", "TestWithoutAssertion.signature"
    ]
    fqn_candidates    = [
        "fqn", "qualifiedName",
        "TestWithoutAssertion", "TestWithoutAssertion.fqn", "TestWithoutAssertion.qualifiedName"
    ]

    class_cols  = [c for c in class_candidates  if c in df_raw.columns]
    method_cols = [c for c in method_candidates if c in df_raw.columns]
    fqn_cols    = [c for c in fqn_candidates    if c in df_raw.columns]

    if not class_cols and not fqn_cols:
        fqn_like = [c for c in df_raw.columns if c.lower().endswith(".fqn") or "qualified" in c.lower()]
        fqn_cols.extend(fqn_like)

    recs = []
    for _, row in df_raw.iterrows():
        cls, mtd = derive_class_and_method(row, class_cols, method_cols, fqn_cols)
        if cls or mtd:
            recs.append({"classFqn": cls or "(unknown)", "methodName": mtd or "(unknown)"})

    df = pd.DataFrame(recs)

    if df.empty:
        print("[info] Test_Without_Assertion.csv loaded but no recognizable columns for class/method.")
    else:
        display(df.head(5))

        # A) Pie: By package (Top 10)
        df["package"] = df["classFqn"].map(package_of)
        by_pkg = df["package"].value_counts().rename_axis("package").reset_index(name="count")
        fig = px.pie(by_pkg.head(10), values="count", names="package",
                     title="Tests without assertion by package (Top 10)", hole=0.45)
        fig.update_layout(height=480, width=720)
        fig.show()

        # B) Bar: Top classes (by count)
        by_class = df["classFqn"].value_counts().rename_axis("classFqn").reset_index(name="count")
        fig = px.bar(by_class.head(25), x="classFqn", y="count", text="count",
                     title="Top classes with tests lacking assertions",
                     color_discrete_sequence=DEFAULT_BAR_COLOR)
        fig.update_traces(textposition="outside", cliponaxis=False)
        fig.update_layout(xaxis_tickangle=-35, height=520, width=1100,
                          xaxis_title="class", yaxis_title="count")
        fig.show()
